# 1. Use Mono Stack to Find Next/Previous Smaller/Larger Element
Monotonic Stack maintaining elements in either increasing or decreasing order.          
It is commonly used to solve the problem of **finding the next and previous greater/smaller element for each element in an array**.            
Without monotonic stack, in a ordinary approach we have to iterate to the left and to the right for each index, thus having a time complexity of O(n^2).   
Using a monotonic stack allow us to do this in **O(n)**.

### Procedure
1. Create a monotonic stack, the stack follows this rule: **elements on top must be strictly larger than all elements below it**
2. iterate through the original array.
3. If `stack.empty` or `nums[stack.peek] < nums[i]`, push `i` onto the stack. Note that our stack stores index rather than value
4. When the top of the stack is larger, **pop all elements that is larger or equals to nums[i]**, then push i onto the stack
5. After we finish iterate through the entire array, pop all the elements that are left on the stack

### We Collect Answer Whenever An Element is Popped
Whenever a element `X` is popped from the stack:
1. Its nearest smaller number on left is the new top of the stack. If the stack is empty after popping `X`, then there is no smaller element on `X`'s left 
2. If it is popped because of we are trying to push a smaller element `Y` on to the stack, `X`'s nearest smaller number on right is `Y`
3. If it is popped during the cleaning stage, there is no smaller element on `X`'s right

### Handling Equivalence (Duplicate Values)
Suppose we are trying to push 5->3 (index->value) on to the stack. What if top of the stack is also 3?(for example 3->3)        
We still pop it and collect answer for 3->3 anyway and fix it later.    

Therefore, after we empty the stack, there might be some places in our answer array that is incorrect due to duplicate value.  

We can fix those answer in one run.         
For each element in the answer array, we check if the `nums[smaller[i][1]] == nums[i]`.    
If this is true, it means that we find a value that is incorrectly popped. To fix this, we just let `smaller[i][1] = smaller[smaller[i][1]][1]`.       
We only check right because in our stack, the top element is strictly larger than bottom element. Therefore our left answer is always correct.


#### **You should design the strategy to handle equivalence value based on the SPECIFIC PROBLEM**
Each problem should has its own way for handling equivalence

### Time Complexity
**O(n)**: Because each element was pushed and popped from the stack only once.         
Thus the average for a single element is O(1) 

### Monotonocity:
If you are looking for **nearest smaller**, stack should be: **large on top, small on bottom.**                

If you are looking for **nearest larger**, stack should be: **small on top, large on bottom.**

In [28]:
class monotonicStack:
    stack = [] 

    def compute(nums):
        n = len(nums)
        #smaller[i][0]: nearest smaller element on the left of i;        smaller[i][1]: nearest smaller element on the right of i;
        smaller = [[-1] * 2 for _ in range(n)]
        for i in range(n):
            # elements on top is strictly larger
            while stack and nums[stack[-1]] >= nums[i]:
                cur = stack.pop()
                smaller[cur][0] = stack.peek if stack else -1
                smaller[cur][1] = i
            stack.append(i)

        # Clearning stage, pop all element from the stack
        while stack:
            cur = stack.pop()
            smaller[cur][0] = stack[-1] if stack else -1
            smaller[cur][1] = -1

        # Fix the errors in our answer due to duplicate value
        for i in range(n - 2, 0, -1):
            if nums[smaller[i][1]] == nums[i]:
                smaller[i][1] = smaller[smaller[i][1]][1]
        

In [5]:
def checkValidString(s):
        low, high = 0, 0  # Track min and max open parenthesis counts

        for char in s:
            if char == '(':
                low += 1
                high += 1
            elif char == ')':
                low -= 1
                high -= 1
            else:  # '*' can act as '(', ')', or ''
                low -= 1  # Treat '*' as ')'
                high += 1  # Treat '*' as '('
            
            if high < 0:  # Too many ')'
                return False
            
            low = max(0, low)  # Low cannot be negative

        return low == 0  # If low > 0, unmatched '(' remain

print(checkValidString("*)("))

1 1
False


---
### Q1. Daily Temperatures (LC.739)
*Given an array of integers temperatures represents the daily temperatures, return an array answer such that answer[i] is the number of days you have to wait after the ith day to get a warmer temperature. If there is no future day for which this is possible, keep answer[i] == 0 instead.*

**Solution:**        
This is even easier than our template, since we only need to find the nearest strictly larger value on the right     

**Handling Equivalence:**        
When we are trying to push an element on to the stack, if `stack[-1] == nums[i]`, we **do NOT pop** the top element and directly push `nums[i]`onto the stack       

This is because we only need to find answer on the right! 
- For example, if our stack is `3, 4, 4`, and `nums[i]` is `5`. We need to pop `3, 4, 4` then push `5`.
- Because both `4` are popped by `5`, their answer on the right will be correct!
- The only thing that is not correct is the left answer, but the problem doesn't ask about it!

In [35]:
class Solution(object):
    def dailyTemperatures(self, temperatures):
        stack = []
        n = len(temperatures)
        ans = [0] * n
        for i in range(n):
            # if top element is the same as nums[i], do not pop
            while stack and temperatures[stack[-1]] < temperatures[i]:
                cur = stack.pop()
                ans[cur] = i - cur
            stack.append(i)
        return ans

---
### Q2. Sum Of Subarray Minimums
*Given an array of integers arr, find the sum of min(b), where b ranges over every (contiguous) subarray of arr. Since the answer may be large, return the answer modulo 10^9 + 7.*

**Solution:**        
For an index `i`, we find the nearest element smaller than `nums[i]` on both side.     
Suppose we find index `l` and `r`.       

Then `nums[i]` is the minimum element for subarrays with starting range in `[l + 1, i]` and ending range in `[i, r - 1]`.      

Thus for index i we got (i - l) * (r - i) subarrays that has `nums[i]` as min.       
Therefore, we get the partial sum by multiplying the number of subarrays found and `nums[i]`.    

For example, suppose i = 5, l = 2, r = 8             
| Index | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
|-------|---|---|---|---|---|---|---|---|---|---|
| Value | . | . | 2 | 7 | 8 | 6 | 8 | 9 | 6 | 7 |



Then we know that these subarrays all has `nums[5] == 6` as min:     
- [3, 5], [3, 6], [3, 7], [4, 5], [4, 6], [4, 7], [5, 5], [5, 6], [5, 7]

Therefore, we calculate the partial sum for each index `i` and add them up together

**Handling Equivalence:**       
We calculate the partial sum when an element is popped. But what if an element is popped by itself?     

Look at the above value, you may think that we mis counted some subarrays, as the subarray [2, 8], [3, 8], [2, 9]... also have the min element of `nums[6]`.
 
But it's acutually find. When we iterate to index `i = 8`, these will be taken care of!

In [71]:
class Solution(object):
    def sumSubarrayMins(self, arr):
        MOD = 1000000007
        stack = []
        n = len(arr)
        ans = 0
        for i in range(n):
            while stack and arr[stack[-1]] >= arr[i]:
                cur = stack.pop()
                r = i
                l = stack[-1] if stack else -1
                ans += (arr[cur] * (cur - l) * (r - cur)) % MOD
            stack.append(i)
        
        while stack:
            cur = stack.pop()
            r = n
            l = stack[-1] if stack else -1
            ans += (arr[cur] * (cur - l) * (r - cur)) % MOD

        return ans % MOD

---
### Q3. Largest Rectangle In Histogram (LC.84)
*Given an array of integers heights representing the histogram's bar height where the width of each bar is 1, return the area of the largest rectangle in the histogram.*

**Solution:**          
Iterate through the array, for each position we consider: If I want to use heights[i] as height, what is the max area I can get?               
If we want to use height[i] as height, we need to find the nearest smaller height on both left and right, so use a monotonic stack.      
Then we find the max area among using difference heights as we iterate through the array.

**Handling Equivalence:**          
Again, similar to Q2, we don't need to manually handle equivalence, even if the current area is not the actual correct result due to equivalence.       
Later when we meet this equivalent value again our algorithm will be able to fix it automatically.

In [89]:
class Solution(object):
    def largestRectangleArea(self, heights):
        n = len(heights)
        stack = []
        ans = 0
        for i in range(n):
            while stack and heights[stack[-1]] >= heights[i]:
                cur = stack.pop()
                l = stack[-1] if stack else -1
                ans = max(ans, (i - l - 1) * heights[cur])
            stack.append(i)
        
        while stack:
            cur = stack.pop()
            l = stack[-1] if stack else -1
            ans = max(ans, (n - l - 1) * heights[cur])
        
        return ans

---
### Q4. Maximal Rectangle
*Given a rows x cols binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.*

**Solution:**        
The 2D version of Q3. Basically the same question.        
We just need to iterate through all the rows, and consider: If I must use row[i] as the base of the rectangle, what is the largest rectangle I can get?      
Then we use matrix compression to change the `submatrix[0, i][j]` into an array `heights[j]`.         
Then this question beceome exactly the same as Q3.

In [91]:
class Solution(object):
    def maximalRectangle(self, matrix):
        m = len(matrix)
        n = len(matrix[0])

        # "Compress" the matrix into array of heights
        # then this problem become the "largest rectangle in histogram" problem
        heights = [[0] * n for _ in range(m)]

        for j in range(n):
            heights[0][j] = int(matrix[0][j])

        for i in range(1, m):
            for j in range(n):
                if matrix[i][j] == '1':
                    heights[i][j] = heights[i - 1][j] + 1
                else:
                    heights[i][j] = 0 

        def largestRectangleArea(heights):
            stack = []
            ans = 0
            n = len(heights)
            
            for i in range(n):
                while stack and heights[stack[-1]] >= heights[i]:
                    cur = stack.pop()
                    l = stack[-1] if stack else -1
                    ans = max(ans, (i - l - 1) * heights[cur])
                stack.append(i)
            
            while stack:
                cur = stack.pop()
                l = stack[-1] if stack else -1
                ans = max(ans, (n - l - 1) * heights[cur])
            
            return ans

        ans = 0
        for i in range(m):
            ans = max(ans, largestRectangleArea(heights[i]))
        return ans     

---
# 2. Use Monotonic Stack As A Container For Possible Answers
Monotonic Stack can also be used to store possible answers.
- When an element is pushed onto the stack, **it "kick out" elements on top of the stack as the current element is a better answer candidate** due to monoticity of the problem
- When an element is popped, it means this element is **no longer useful and does not participate in future answer finding** process anymore

---
### Q1. Maximum Ramp Width (LC.962)
*A ramp in an integer array nums is a pair (i, j) for which i < j and nums[i] <= nums[j]. The width of such a ramp is j - i.*
*Given an integer array nums, return the maximum width of a ramp in nums. If there is no ramp in nums, return 0.*

**Solution:**                 
We use a monotonic stack in which elements on top is strictly smaller than elements in bottom.         

We iterate from left to right, and for an element `nums[i]`, we only put it onto the stack if it is smaller than `stack[-1]`
- This is because we are iterating from left to right, therefore if `nums[i] > stack[-1]` it is **not a possible answer** because stack[-1] is further on the left and smaller at the same time
- What we pushed on to the stack is the candidate first element in the answer pair

When we finish filling the stack with possible answers, we iterate through the array again but from the right. This time we are looking for possible second elements in the answer pair.   
- If the top of the stack doesn't work with the current element, we skip this element and move to the left
- If the top of the stack work, we pop the top of the stack, update answer, and check with the next top element!
- We can pop an element since once it got paired with an element from right, it will no longer participate in the process of answer finding, since all elements below it on the stack are better possible candidates!(since the elements below are furthur on the left)

In [7]:
class Solution(object):
    def maxWidthRamp(self, nums):
        n = len(nums)
        stack = []
        stack.append(0)
        for i in range(n):
            if nums[i] < nums[stack[-1]]:
                stack.append(i)
        ans = 0
        for i in range(n - 1, -1, -1):
            while stack and nums[i] >= nums[stack[-1]]:
                cur = stack.pop()
                ans = max(ans, i - cur)
        return ans

---
### Q2. Remove Duplicate Letters (LC.316)
*Given a string s, remove duplicate letters so that every letter appears once and only once. You must make sure your result is 
the smallest in lexicographical order among all possible results.*

**Solution:**          
This time, we directly store our answer string in a monotonic stack.           
We first iterate through the array to get the count of each character, then start pushing letters on to the stack.          
Elements on top should be strictly larger than elements in the bottom.  

In [6]:
class Solution(object):
    def removeDuplicateLetters(self, s):
        lettersCnt = defaultdict(int)
        collected = set()                               # Keep track of what letters are already included in the answer
        for ch in s:
            lettersCnt[ch] += 1
        stack = []
        for ch in s:
            if ch not in collected:                    # If letter already collected, skip
                # Note that we can only pop an element if we are able to add it back later i.e. the letter will also occur later
                while stack and stack[-1] >= ch and lettersCnt[stack[-1]] >= 1:
                    cur = stack.pop()
                    collected.remove(cur)
                stack.append(ch)
                collected.add(ch)
            lettersCnt[ch] -= 1
        return "".join(stack)

---
### Q3. Count Submatrices With All Ones (LC.1504)
*Given an m x n binary matrix mat, return the number of submatrices that have all ones.*